In [1]:
import pandas as pd
import jieba
import string

from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
df = pd.read_csv('品類資料集\cat_train_v2.csv')
df = df[['name', 'product']]
'''已知問題資料: 9691, 法文, 日文'''

'已知問題資料: 9691, 法文, 日文'

In [3]:
df = df[df['name'] != '#NAME?']
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)
# df.set_index(['index'], inplace=True)

In [4]:
with open('stop.txt', 'r', encoding='utf-8') as f:
    stop = [i.strip() for i in f.readlines()]

In [5]:
def remove_things(text):
    text = "".join([i for i in text if i not in stop])               # 移除停止詞
    text = "".join([i for i in text if i not in string.punctuation]) # 移除標點
    text = "".join([i for i in text if i not in string.whitespace])  # 移除空格
    return text

def strQ2B(ustring):
    """把字串全形轉半形"""
    ss = []
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全形空格直接轉換
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全形字元（除空格）根據關係轉化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss.append(rstring)
    return ''.join(ss)

In [6]:
df['name'] = df['name'].apply(lambda x: x.lower())
df['name'] = df['name'].apply(lambda x: remove_things(x))
df['name'] = df['name'].apply(lambda x: strQ2B(x))

In [9]:
df = df[df['name'] != '']
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)

In [10]:
df

,name,product
0,r視舒坦支裝工淚液點眼液ml支入,人工淚液
1,優麗舒點眼液ml,人工淚液
2,r視舒坦支裝工,人工淚液
3,rohto乾眼淚液,人工淚液
4,r參天柔潤工淚液點眼液ml入,人工淚液
...,...,...
92289,mdmmd微醺心機漸層腮紅玫瑰瑪格麗特,腮紅
92290,mdmmd微醺心機漸層腮紅橙酒,腮紅
92291,倫敦芮魅叛逆玩色腮紅盤微醺酒紅,腮紅
92292,palladio柔幻彩貝殼腮紅微醺蜜桃,腮紅


In [11]:
cut_all_false = []
cut_all_true = []
HMM_false = []
HMM_true = []
paddle_false = []
paddle_true = []
all_false = []
all_true = []

for i in range(len(df)):
    cut_all_false.append(','.join(jieba.cut(df.name[i], cut_all=False, HMM=True, use_paddle=True)))
    cut_all_true.append(','.join(jieba.cut(df.name[i], cut_all=True, HMM=False, use_paddle=False)))
    HMM_false.append(','.join(jieba.cut(df.name[i], cut_all=True, HMM=False, use_paddle=True)))
    HMM_true.append(','.join(jieba.cut(df.name[i], cut_all=False, HMM=True, use_paddle=False)))
    paddle_false.append(','.join(jieba.cut(df.name[i], cut_all=True, HMM=True, use_paddle=False)))
    paddle_true.append(','.join(jieba.cut(df.name[i], cut_all=False, HMM=False, use_paddle=True)))
    all_false.append(','.join(jieba.cut(df.name[i], cut_all=False, HMM=False, use_paddle=False)))
    all_true.append(','.join(jieba.cut(df.name[i], cut_all=True, HMM=True, use_paddle=True)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dddru\AppData\Local\Temp\jieba.cache
Loading model cost 0.882 seconds.
Prefix dict has been built successfully.


In [ ]:
jb = pd.DataFrame({'cut_all_false':cut_all_false,
                    'cut_all_true': cut_all_true,
                    'HMM_false': HMM_false,
                    'HMM_true': HMM_true,
                    'paddle_false': paddle_false,
                    'paddle_true': paddle_true,
                    'all_false': all_false,
                    'all_true': all_true})

In [ ]:
df = pd.concat([df, jb], axis=1)
mlb = MultiLabelBinarizer()
mlb_result = mlb.fit_transform([str(df.loc[i, 'product']).split(';') for i in range(len(df))])
df_final = pd.concat([df.drop(['product'], axis=1), pd.DataFrame(mlb_result, columns=list(mlb.classes_))], axis=1)
df_final.to_csv('preprocessed_jieba_cat_train.csv', index=False)

In [5]:
def segmentation(df):
    result = []
    for line in df:
        result.append([x for x in jieba.cut(line)])
    
    return result